## <font color=orange>Extracting Subduction Zone Kinematics & Parameters (pyGPlates method)</font>

#### <font color=blue>Notebook 1</font>

**This notebook can be used to extract subduction zone kinematics and parameters from a chosen plate model. These include:**
- overriding and subducting plate ID's
- arc length
- lat and lon
- time
- convergence rate and obliquity
- migration rate and obliquity 
- age of the subducting seafloor
- distance to nearest continent

**It illustrates usage of the updated subduction zone kinematics pygplates method, using the Müller et al AREPS model:**

- implemented to work directly on the plate model gpml files (whereas before, you had to export the resolved topologies at 1Myr increments first)
- this notebook also interpolates the age of the subducting plate age from age grids if provided
- within the notebook data are stored in pandas dataframes, and there is an illustration of how to export them to both csv or hdf5 files

This was the first in a series of notebooks used to generate the data presened in the following manuscript:

*'Subduction history reveals Cretaceous slab superflux as a possible cause for the mid-Cretaceous plume pulse and superswell events'* by Madison East, R Dietmar Müller, Simon Williams, Sabin Zahirovic and Christian Heine


(Notebook written by Simon Williams with minor modifications made by Madison East)


In [ ]:
# This cell loads in the required files for the chosen plate model and defines some required functions #
#########################################################################################################

import pygplates
import subduction_convergence as sc #(required code stored in a .py file)
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib import gridspec
from netCDF4 import Dataset
import scipy.interpolate as spi
import inpaint
import pandas as pd

%matplotlib inline

###################################################################
# Define Input files for Muller 2016 AREPS model (or chosen model)
###################################################################
# Specify the name of the GPlates rotation file
rotation_filename = '../Data/Global_EarthByte_230-0Ma_GK07_AREPS.rot'

# input topologies to be used with the subduction_convergence script
input_topology_filename = ['../Data/Global_EarthByte_230-0Ma_GK07_AREPS_PlateBoundaries.gpmlz',\
             '../Data/Global_EarthByte_230-0Ma_GK07_AREPS_Topology_BuildingBlocks.gpmlz']

# The list array here specifies the age grid files, 
# assuming that the age grids are all together in a folder, with common filenames that
# differ only in the integer number somewhere in the filename that corresponds to the age in Ma
# The two parts of the list specify the parts of the filename before and after the number
# For example:
# ['/path_to_age_grids/age_grids_at_time_','_Ma.grd]
agegrid_filename = ['agegrids/Muller_etal_2019_Tectonics_v2.0_netCDF/Muller_etal_2019_Tectonics_v2.0_AgeGrid-','.nc']

# Static polygons to determine whether subduction segment is adjacent to continent or not
static_polygon_filename = '../Data/StaticPolygons/Global_EarthByte_GPlates_PresentDay_StaticPlatePolygons_2015_v1.shp'
static_polygon_features = pygplates.FeatureCollection(static_polygon_filename)
continental_polygon_features = []
for feature in static_polygon_features:
    if feature.get_feature_type() == pygplates.FeatureType.gpml_closed_continental_boundary:
        continental_polygon_features.append(feature)


######################################################
rotation_model = pygplates.RotationModel(rotation_filename)

# specify time range and resolution for plots
threshold_sampling_distance_radians = np.radians(0.5)

# Define the time snapshots at which to get the subduction zone properties
min_time = 0.
max_time = 230.
time_step = 1.

# Set the delta time for velocity calculations
velocity_delta_time = 1.

# Typically the achor plate id should be 0
anchor_plate_id = 0


#################################################
# Included functions
#################################################
def sample_grid_using_scipy(x,y,grdfile):
    
    with Dataset(grdfile, 'r', format='NETCDF4') as data:
        try:
            lon = np.asarray(data['x'])
            lat = np.asarray(data['y'])
        except (KeyError, IndexError):
            lon = np.asarray(data['lon'])
            lat = np.asarray(data['lat'])
    
        Zg = np.asarray(data['z'])
    
    test = inpaint.fill_ndimage(Zg)
    
    lut=spi.RectBivariateSpline(lon,lat,test.T)
    result = []
    for xi,yi in zip(x,y):
        result.append(lut(xi, yi)[0][0])
            
    return result


def get_nearest_continental_polygon(subduction_data,continental_polygons):

    nearest_continent_plate_id = []
    distance_to_nearest_continent = []
    
    for index,row in subduction_data.iterrows():
        
        min_distance_to_all_features = np.radians(180)
        nearest_continent = None        
        
        seed_point = pygplates.PointOnSphere(row['lat'], row['lon'])
        
        for polygon in continental_polygons:
            if polygon is not None:
                min_distance_to_feature = pygplates.GeometryOnSphere.distance(
                    polygon.get_reconstructed_geometry(),
                    seed_point,
                    min_distance_to_all_features,
                    geometry1_is_solid=True)

                # If the current geometry is nearer than all previous geometries then
                # its associated feature is the nearest feature so far.
                if min_distance_to_feature is not None:
                    min_distance_to_all_features = min_distance_to_feature
                    nearest_continent = polygon.get_feature().get_reconstruction_plate_id()
                    
        nearest_continent_plate_id.append(nearest_continent)
        distance_to_nearest_continent.append(min_distance_to_all_features*pygplates.Earth.mean_radius_in_kms)

    return nearest_continent_plate_id,distance_to_nearest_continent

### Number crunching section

In [ ]:
%%capture --no-stdout
# Create a dataframe that contains all subduction data for full sequence of time steps

# Data frame template defining the column names
DataFrameTemplate = ('lon','lat','conv_rate','conv_obliq','migr_rate',
                     'migr_obliq','arc_length','arc_azimuth',
                     'subducting_plate','overriding_plate','time')

# Create an empty dataframe to concatenate results to
df_AllTimes = pd.DataFrame(columns=DataFrameTemplate)

# list of reconstruction times
reconstruction_times = np.arange(min_time,max_time+1,time_step)

# Iterate over time steps
for reconstruction_time in reconstruction_times:
        
    # Get the subduction kinematics database 
    subduction_data = []
    output_data = sc.subduction_convergence(
                rotation_filename,
                input_topology_filename,
                threshold_sampling_distance_radians,
                reconstruction_time,
                velocity_delta_time,
                anchor_plate_id)

    # Make a flat list of subduction stats to input into the proximity test
    for data in output_data:
        subduction_data.append(data+(reconstruction_time,))
    
    # convert list array to dataframe
    df = pd.DataFrame(subduction_data, columns = DataFrameTemplate)
    
    ######################
    # Age grid sampling
    grdfile = '%s%0.0f%s' % (agegrid_filename[0],reconstruction_time,agegrid_filename[1])
    print('working on file %s' % grdfile)
    sample_result = sample_grid_using_scipy(df['lon'],df['lat'],grdfile)
    
    # Append interpolate seafloor ages to dataframe
    df['SeafloorAge'] = pd.Series(sample_result)
    
    
    ######################
    # Find nearest continental polygon
    reconstructed_continental_polygons = []
    pygplates.reconstruct(continental_polygon_features,
                          rotation_model,
                          reconstructed_continental_polygons,
                          reconstruction_time,
                          anchor_plate_id=anchor_plate_id)
    pid,dist = get_nearest_continental_polygon(df,reconstructed_continental_polygons)
    df['NearestContinentPID'] = pd.Series(pid)
    df['DistanceToContinent'] = pd.Series(dist)

    # append dataframe 
    #df_AllTimes = df_AllTimes.append(df)
    df_AllTimes = pd.concat([df_AllTimes, df], ignore_index=True)
    

### Options for saving/loading results to/from file

In [ ]:
# Save to csv file (prefered file format)
df_AllTimes.to_csv('SubductionTable_clean_%0.0f_%0.0fMa.csv' % (min_time,max_time)) 

# Save to an hdf5 file
#df_AllTimes.to_hdf('SubductionTable_%0.0f_%0.0fMa.h5' % (min_time,max_time),'SubductionTable') 

# Example of loading previously calculated results from a hdf5 file as created using the line above
#df_AllTimes = pd.read_hdf('SubductionTable_0_230Ma.h5')

#plt.scatter(df.lon,df.lat,c=df.DistanceToContinent,edgecolors='')
#plt.show()


In [ ]:
# Display a preveiw of the data we just created
df_AllTimes